In [1]:
#import pynq libraries
from pynq import Overlay
from pynq import allocate
#import matplotlib.pyplot as plt
import math
import time
import numpy as np

In [3]:
#show documentation
#testmem_ip?

In [4]:
#show testmem ip register maps
#testmem_ip.register_map

In [5]:
Mbyte=20

overlay =Overlay("./design_64_wrapper.xsa")
testmem_ip=overlay.kernel_0
datatype=64
#show documentation
#testmem_ip?
#show testmem ip register maps
#testmem_ip.register_map

#inizialize data
max_data=int((Mbyte*1024*1024*8)/datatype)
#FPGA
data= np.zeros((max_data))
#data buffer creation for FPGA
buf_data = allocate(max_data,np.uint64)
buf_data[:]=data[:]
buf_data.flush()
testmem_ip.write(0x10,buf_data.physical_address)

#kernel function attivation and time calculation
w_time_64= np.zeros(20)
#decide Mbyte intervals 
sud=int(Mbyte/20)
data_n=int((sud*1024*1024*8)/datatype)
#starting write time calculation
p=1
while p<=20:
    #100 subtests to create an avarage time
    c=0
    w_sum_64=0
    while c<100:
        data_n=int((sud*1024*1024*8)/datatype)
        data_n=data_n*(p)
        testmem_ip.write(0x1c,data_n)
        testmem_ip.write(0x00,1)
        #start counting
        start=time.time()
        #continue until it end 
        while testmem_ip.read(0x00)& 0x04!=0x04:
            pass
        #finish counting
        stop=time.time()
        w_sum_64=w_sum_64+(stop-start)*1000
        c=c+1
    w_time_64[p-1]=w_sum_64/100
    p=p+1
print(data_n)

2621440


In [6]:
print(w_time_64)

[ 1.81659222  3.61887932  5.42091608  7.24513292  9.02611017 10.82888365
 12.63078928 14.43214417 16.23469591 18.0366993  19.83901024 21.64115667
 23.44431639 25.24610043 27.04963207 28.85080576 30.65317631 32.45563745
 34.25671816 36.05928898]


In [ ]:
Mbyte=20
for n in range(2):
    #kernel function's IP made for write from the FPGA
    if n==0:
        overlay =Overlay("./design_64_wrapper.xsa")
        testmem_ip=overlay.kernel_0
        datatype=64
    if n==1:
        overlay =Overlay("./design_1_128_wrapper.xsa")
        testmem_ip=overlay.kernel_0
        datatype=128
    if n==2:
        overlay =Overlay("./design_1_256_wrapper.xsa")
        testmem_ip=overlay.kernel_0
        datatype=256
    
    #inizialize data
    max_data=int((Mbyte*1024*1024*8)/datatype)
    #FPGA
    data= np.zeros((max_data))
    #data buffer creation for FPGA
    buf_data = allocate(max_data,np.uint64)
    buf_data[:]=data[:]
    buf_data.flush()
    testmem_ip.write(0x10,buf_data.physical_address)
    
    #kernel function attivation and time calculation
    w_time_64= np.zeros(20)
    w_time_128=np.zeros(20)
    w_time_256=np.zeros(20)
    #decide Mbyte intervals 
    sud=int(Mbyte/20)
    data_n=int((sud*1024*1024*8)/datatype)
    #starting write time calculation
    p=1
    while p<=20:
        #100 subtests to create an avarage time
        c=0
        w_sum_64=0
        w_sum_128=0
        w_sum_256=0
        while c<100:
            data_n=int((sud*1024*1024*8)/datatype)
            data_n=data_n*(p)
            testmem_ip.write(0x1c,data_n)
            testmem_ip.write(0x00,1)
            #start counting
            start=time.time()
            #continue until it end 
            while testmem_ip.read(0x00)& 0x04!=0x04:
                pass
            #finish counting
            stop=time.time()
            if n==0:
                w_sum_64=w_sum_64+(stop-start)*1000
            if n==1:
                w_sum_128=w_sum_128+(stop-start)*1000
            if n==2:
                w_sum_256=w_sum_256+(stop-start)*1000
            c=c+1
        if n==0:
            w_time_64[p-1]=w_sum_64/100
        if n==1:
            w_time_128[p-1]=w_sum_128/100
        if n==2:        
            w_time_256[p-1]=w_sum_256/100
        p=p+1
    print(data_n)

2621440


In [9]:
print(w_time_64)
print(w_time_128)
print(w_time_256)


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[ 1.52101755  2.96378851  4.43900824  5.91397285  7.38945484  8.87286425
 10.33741713 11.81376219 13.28603983 14.76175547 16.23687983 17.71109104
 19.18617487 20.66010237 22.13450432 23.60899925 25.08298635 26.55930042
 28.03374052 29.50743914]
